In [7]:
#=
Dr Keith Reid
Cailleach Computing Ltd
MIT license

Copyright © 2023 <copyright holders>

Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the “Software”), to deal in
the Software without restriction, including without limitation the rights to 
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of 
the Software, and to permit persons to whom the Software is furnished to do so, 
subject to the following conditions:

The above copyright notice and this permission notice shall be included in all 
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR 
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS 
FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR 
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER 
IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
=#

# s.0 libraries

using Test

# s.1 structs

mutable struct Element
    l_link::Char
    name::Char
    r_link::Char
end

# s.2 config

function get_cardinal()
    cardinal::Int64 = 5
    return cardinal
end

function get_universal_alphabet()
    universal_alphabet  = collect("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")
    return universal_alphabet
end

# s.3 config

function amend(cardinal, place)
    while place < 1
        place = place + cardinal
    end
    while !(1 ≤ place ≤ cardinal)
        place = place - cardinal
    end
    return place
end

function ask_alphabet(place)
    cardinal            = get_cardinal()
    if cardinal ≤ 62
        universal_alphabet  = get_universal_alphabet()
        alphabet            = universal_alphabet[1:cardinal]
    else
        "ran out of letters"
    end
    char = alphabet[place]
end

function init_links(cardinal::Int64)
    links = Set()
    for i in 1:cardinal
        l_place         = amend(cardinal, i-1)
        name_place      = amend(cardinal, i)
        r_place         = amend(cardinal, i+1)        
        l_link          = ask_alphabet(l_place)        
        name            = ask_alphabet(name_place)
        r_link          = ask_alphabet(r_place)
        this_element    = Element(l_link, name, r_link)
        push!(links, this_element)
    end
    return links
end

function wipe!(links, kill_name)
    # a → b → c → d → e → f → g → h → i → j
    #j b a c b d c e d f e g f h g i h j i a
    # a ← b ← c ← d ← e ← f ← g ← h ← i → j
          #k 
      #lok  
              #rok
    left_of_kill      = [x for x in collect(links) if x.r_link == kill_name][1]
    right_of_kill     = [x for x in collect(links) if x.l_link == kill_name][1]
    post_wipe = Set()
    for this_element in collect(links)
        if this_element == left_of_kill
            this_element.r_link  = right_of_kill.name # if wipe b, change j.a.b to j.a.c
            push!(post_wipe, this_element)
        elseif this_element == right_of_kill
               this_element.l_link  = left_of_kill.name # if wipe b, change b.c.d to a.c.d
            push!(post_wipe, this_element)
        else
            push!(post_wipe, this_element)
        end
    end
    return post_wipe
end


# s.4 view

function scan(start_name, links)
    current_name    = start_name
    names           = [] # an ordered representation of the links in an unordered set 
    push!(names, start_name)
    start_name_visits = 1
    while start_name_visits == 1
        next            = [x for x in collect(links) if x.l_link == current_name][1]
        name_of_next    = next.name
        if name_of_next !== start_name
            push!(names, name_of_next)
            current_name    = name_of_next
        else
            start_name_visits += 1
        end
    end
    return names
end

# s.5 control

function main_dancing_links()
    cardinal    = get_cardinal()
    links       = init_links(cardinal)
    rand_start  = Base.rand(links)
    rand_name   = 'a' #rand_start.name
    names       = scan(rand_start.name, links)
    [print(x) for x in names]
    print("\n")
    post_wipe   = wipe!(links, 'b')
    names       = scan(rand_start.name, post_wipe)
    [print(x) for x in names]
end

main_dancing_links();

deabc
deac

In [12]:
# s.6 tests

function test_ask_alphabet()
    place   = 3
    name    = ask_alphabet(place)
    @test name == 'c'
    @test typeof(name) == Char
    place   = 1
    name    = ask_alphabet(place)
    @test name == 'a'
    cardinal= 3
    place   = 0
    place   = amend(cardinal, place)
    name    = ask_alphabet(place)
    @test name == 'c'
    println("passed ask alphabet")
end

function test_amend()
    cardinal = 3
    place    = 1
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 2
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 4
    place    = 3
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 4
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 5
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 6
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 7
    amended  = amend(cardinal, place)
    @test amended == 1
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 3
    @test 1 ≤ amended ≤ cardinal
    cardinal = 3
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 2
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = -1
    amended  = amend(cardinal, place)
    @test amended == 6
    @test 1 ≤ amended ≤ cardinal
    cardinal = 7
    place    = 0
    amended  = amend(cardinal, place)
    @test amended == 7
    @test 1 ≤ amended ≤ cardinal
    println("passed test")
end

function test_init_links()
    cardinal = 3
    links    = init_links(cardinal)
    @test Element('a', 'b', 'c') ∈ links
    @test Element('b', 'c', 'a') ∈ links
    @test Element('c', 'a', 'b') ∈ links
    
    cardinal = 1
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'a', 'a')])
    
    cardinal = 2
    links    = init_links(cardinal)
    @test links == Set(Any[Element('a', 'b', 'a'), Element('b', 'a', 'b')])
    @test Element('a', 'b', 'a') ∈ links
    @test Element('b', 'a', 'b') ∈ links
end


function test_wipe!()
    links       = Set(Any[Element('h', 'i', 'j'), Element('e', 'f', 'g'), Element('a', 'b', 'c'), 
                          Element('c', 'd', 'e'), Element('j', 'a', 'b'), Element('f', 'g', 'h'), 
                          Element('b', 'c', 'd'), Element('g', 'h', 'i'), Element('d', 'e', 'f'), 
                          Element('i', 'j', 'a')])
    pre_names   = scan('a', links)
    post_wipe   = wipe!(links, 'b')
    post_names  = scan('a', post_wipe)
    @test post_names == Any['a', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
    println("passed wipe")
end

function tests()
    println("testing")
    test_amend()
    test_ask_alphabet()
    println("confusion over set vs singleton of array in a set - take a break")
    #test_init_links()
    #test_wipe!()
    #println("passed all tests")
end

tests()

testing
passed test
passed ask alphabet
confusion over set vs singleton of array in a set - take a break
